## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy.stats as stats

## IMPORT DATA

In [2]:
CreditConsumptionData=pd.read_excel("C://Users//aazar//dwhelper//ml project 1//CreditConsumptionData.xlsx")
CustomerBehaviorData=pd.read_excel("C://Users//aazar//dwhelper//ml project 1//CustomerBehaviorData.xlsx")
CustomerDemographics=pd.read_excel("C://Users//aazar//dwhelper//ml project 1//CustomerDemographics.xlsx")

In [3]:
df1=pd.merge(CreditConsumptionData,CustomerBehaviorData,on="ID",how="inner")
df=pd.merge(df1,CustomerDemographics,on="ID",how="inner")

In [4]:
df

,ID,cc_cons,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,...,emi_active,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
0,12554,20014.0,24893.000,378.00,10288.00,29664.00,16291.40,11432.00,2.0,26,...,1674.09,current,M,35,MEDIUM,15.0,1,708.0,0,17.0
1,17645,10173.0,18941.620,966.00,20672.00,287.00,4217.00,885.00,1.0,7,...,13043.34,current,M,35,HIGH,15.0,6,123.0,0,14.0
2,7604,16095.0,5678.870,2724.00,1964.50,3933.11,23956.25,5168.00,43.0,32,...,25375.27,current,M,55,MEDIUM,24.5,1,802.0,1,3.0
3,1590,7707.0,30489.500,1236.00,12609.88,9138.14,17521.00,13650.30,53.0,1,...,3544.33,current,F,29,MEDIUM,4.5,6,867.0,1,4.0
4,16556,96408.0,7441.400,6906.04,4364.00,1939.00,2121.00,6829.18,67.0,47,...,12780.44,current,M,34,MEDIUM,12.6,4,802.0,1,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,12512,NaN,4641.500,5979.58,6008.61,4410.00,41743.00,1884.00,26.0,1,...,7870.88,current,M,35,MEDIUM,13.5,1,523.0,0,7.0
19996,14224,NaN,4994.450,2805.37,23214.00,5870.00,1041.00,1008.73,37.0,56,...,3668.30,current,M,53,MEDIUM,9.9,7,723.0,1,2.0
19997,18740,NaN,3262.145,2871.00,7335.25,11774.04,4130.00,1413.00,12.0,77,...,5450.34,current,M,62,HIGH,37.8,8,863.0,1,16.0
19998,18134,NaN,12824.010,4866.00,12242.48,284.00,15132.77,37911.00,67.0,1,...,59.29,current,M,34,MEDIUM,12.6,4,882.0,0,13.0


In [5]:
# Create Data audit Report for continuous variables
def continuous(x):
    return pd.Series([x.sum(),x.count(),x.isna().sum(),x.var(),x.std(),x.mean(),x.median(),x.min()
                     ,x.quantile(0.01),x.quantile(0.05),x.quantile(0.75)
                     ,x.quantile(0.10),x.quantile(0.25),x.quantile(0.50)
                     ,x.quantile(0.90),x.quantile(0.95),x.quantile(0.99),x.max()],
                    index=["sum","count","nmiss","var","std","mean","median","min","p1"
                          ,"p5","p10","p25","p50","p90","p95","p99","max"])

In [6]:
# Create Data audit Report for categorical variables
def categorical(x):
    mode=x.value_counts().sort_value(ascending=False)[0:1].reset_index()
    return pd.Series([x.count,x.isna().sum(),mode.loc[0,0],mode.iloc[0,1],
                      round(mode.iloc[0,1]*100/x.count(),2)],
                    index=['N', 'NMISS', 'MODE', 'FREQ', 'PERCENT'])

In [7]:
# Missing value imputation for categorical and continuous variables
def missing_imputer(x,stat="mean"):
    if x.dtypes=="float64" or x.dtypes=="int64":
        x=x.fillna(x.mean()) if stat=="mean" else x.fillna(x.median())
    else:
        x=x.fillna(x.mode()).iloc[0]
        return x

In [8]:
df_existing=df

In [9]:
df_existing

,ID,cc_cons,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,...,emi_active,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
0,12554,20014.0,24893.000,378.00,10288.00,29664.00,16291.40,11432.00,2.0,26,...,1674.09,current,M,35,MEDIUM,15.0,1,708.0,0,17.0
1,17645,10173.0,18941.620,966.00,20672.00,287.00,4217.00,885.00,1.0,7,...,13043.34,current,M,35,HIGH,15.0,6,123.0,0,14.0
2,7604,16095.0,5678.870,2724.00,1964.50,3933.11,23956.25,5168.00,43.0,32,...,25375.27,current,M,55,MEDIUM,24.5,1,802.0,1,3.0
3,1590,7707.0,30489.500,1236.00,12609.88,9138.14,17521.00,13650.30,53.0,1,...,3544.33,current,F,29,MEDIUM,4.5,6,867.0,1,4.0
4,16556,96408.0,7441.400,6906.04,4364.00,1939.00,2121.00,6829.18,67.0,47,...,12780.44,current,M,34,MEDIUM,12.6,4,802.0,1,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,12512,NaN,4641.500,5979.58,6008.61,4410.00,41743.00,1884.00,26.0,1,...,7870.88,current,M,35,MEDIUM,13.5,1,523.0,0,7.0
19996,14224,NaN,4994.450,2805.37,23214.00,5870.00,1041.00,1008.73,37.0,56,...,3668.30,current,M,53,MEDIUM,9.9,7,723.0,1,2.0
19997,18740,NaN,3262.145,2871.00,7335.25,11774.04,4130.00,1413.00,12.0,77,...,5450.34,current,M,62,HIGH,37.8,8,863.0,1,16.0
19998,18134,NaN,12824.010,4866.00,12242.48,284.00,15132.77,37911.00,67.0,1,...,59.29,current,M,34,MEDIUM,12.6,4,882.0,0,13.0


In [10]:
df_existing.head()

,ID,cc_cons,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,...,emi_active,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
0,12554,20014.0,24893.00,378.00,10288.00,29664.00,16291.40,11432.00,2.0,26,...,1674.09,current,M,35,MEDIUM,15.0,1,708.0,0,17.0
1,17645,10173.0,18941.62,966.00,20672.00,287.00,4217.00,885.00,1.0,7,...,13043.34,current,M,35,HIGH,15.0,6,123.0,0,14.0
2,7604,16095.0,5678.87,2724.00,1964.50,3933.11,23956.25,5168.00,43.0,32,...,25375.27,current,M,55,MEDIUM,24.5,1,802.0,1,3.0
3,1590,7707.0,30489.50,1236.00,12609.88,9138.14,17521.00,13650.30,53.0,1,...,3544.33,current,F,29,MEDIUM,4.5,6,867.0,1,4.0
4,16556,96408.0,7441.40,6906.04,4364.00,1939.00,2121.00,6829.18,67.0,47,...,12780.44,current,M,34,MEDIUM,12.6,4,802.0,1,7.0


In [11]:
df_existing.describe()

,ID,cc_cons,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,...,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active,age,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
count,20000.000000,15000.000000,20000.000000,20000.000000,1.999900e+04,20000.000000,20000.000000,19999.000000,19999.000000,20000.000000,...,20000.000000,20000.000000,2.000000e+04,1.999900e+04,20000.000000,20000.000000,20000.000000,19999.000000,20000.000000,19997.000000
mean,9999.500000,55157.729867,17856.990922,8130.453414,1.618836e+04,7702.427191,12135.806021,7109.708341,41.611631,33.569100,...,19.905450,44.529100,5.284985e+04,1.539887e+04,47.672300,15.217250,5.496050,598.959348,0.718000,11.044757
std,5773.647028,97670.972252,25158.878234,14572.188829,2.651068e+04,17892.731800,18790.984789,13313.935609,97.835475,50.316925,...,27.055323,35.080453,1.343175e+05,1.202541e+05,30.359771,20.481455,2.870452,220.105230,0.449985,5.480552
min,0.000000,0.000000,253.000000,19.350000,1.570000e+02,22.000000,65.000000,62.000000,1.000000,1.000000,...,1.000000,0.000000,6.000000e-02,0.000000e+00,25.000000,0.500000,1.000000,123.000000,0.000000,2.000000
25%,4999.750000,6768.000000,4377.000000,1624.650000,3.635910e+03,1634.260000,2999.075000,1361.100000,5.000000,8.000000,...,4.000000,12.000000,1.896450e+04,1.545940e+03,33.000000,5.100000,3.000000,424.000000,0.000000,6.000000
50%,9999.500000,13364.000000,10214.642500,3964.750000,8.635230e+03,3791.720000,6656.800000,3326.000000,10.000000,23.000000,...,7.000000,37.000000,3.025000e+04,3.934660e+03,40.000000,9.900000,6.000000,623.000000,1.000000,11.000000
75%,14999.250000,20105.750000,21189.000000,9339.020000,1.865531e+04,8263.590000,14195.727500,7458.820000,44.000000,43.000000,...,21.000000,73.000000,5.344900e+04,9.376390e+03,53.000000,18.000000,8.000000,799.000000,1.000000,16.000000
max,19999.000000,408382.000000,441031.330000,229255.000000,1.117246e+06,911223.490000,382914.100000,233384.000000,531.000000,601.000000,...,117.000000,133.000000,5.200000e+06,7.447125e+06,223.000000,203.000000,10.000000,974.000000,1.000000,20.000000


## Droping the ID column

In [12]:
df_existing.drop(columns="ID",axis=1,inplace=True)

In [13]:
df_existing

,cc_cons,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,...,emi_active,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
0,20014.0,24893.000,378.00,10288.00,29664.00,16291.40,11432.00,2.0,26,10,...,1674.09,current,M,35,MEDIUM,15.0,1,708.0,0,17.0
1,10173.0,18941.620,966.00,20672.00,287.00,4217.00,885.00,1.0,7,13,...,13043.34,current,M,35,HIGH,15.0,6,123.0,0,14.0
2,16095.0,5678.870,2724.00,1964.50,3933.11,23956.25,5168.00,43.0,32,102,...,25375.27,current,M,55,MEDIUM,24.5,1,802.0,1,3.0
3,7707.0,30489.500,1236.00,12609.88,9138.14,17521.00,13650.30,53.0,1,50,...,3544.33,current,F,29,MEDIUM,4.5,6,867.0,1,4.0
4,96408.0,7441.400,6906.04,4364.00,1939.00,2121.00,6829.18,67.0,47,82,...,12780.44,current,M,34,MEDIUM,12.6,4,802.0,1,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,NaN,4641.500,5979.58,6008.61,4410.00,41743.00,1884.00,26.0,1,4,...,7870.88,current,M,35,MEDIUM,13.5,1,523.0,0,7.0
19996,NaN,4994.450,2805.37,23214.00,5870.00,1041.00,1008.73,37.0,56,50,...,3668.30,current,M,53,MEDIUM,9.9,7,723.0,1,2.0
19997,NaN,3262.145,2871.00,7335.25,11774.04,4130.00,1413.00,12.0,77,1,...,5450.34,current,M,62,HIGH,37.8,8,863.0,1,16.0
19998,NaN,12824.010,4866.00,12242.48,284.00,15132.77,37911.00,67.0,1,4,...,59.29,current,M,34,MEDIUM,12.6,4,882.0,0,13.0


## Separating the numerical and categorical columns

In [14]:
numeric=[i for i in dict(df_existing.dtypes) if dict(df_existing.dtypes)[i] in ["float64","int64","float32","int32"]]
categorical_col=[i for i in dict(df_existing.dtypes) if dict(df_existing.dtypes)[i] in ["object"]]

In [15]:
print(numeric)
print(categorical_col)

['cc_cons', 'cc_cons_apr', 'dc_cons_apr', 'cc_cons_may', 'dc_cons_may', 'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun', 'dc_count_apr', 'dc_count_may', 'dc_count_jun', 'card_lim', 'personal_loan_active', 'vehicle_loan_active', 'personal_loan_closed', 'vehicle_loan_closed', 'investment_1', 'investment_2', 'investment_3', 'investment_4', 'debit_amount_apr', 'credit_amount_apr', 'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr', 'debit_amount_may', 'credit_amount_may', 'credit_count_may', 'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun', 'credit_amount_jun', 'credit_count_jun', 'debit_count_jun', 'max_credit_amount_jun', 'emi_active', 'age', 'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code', 'NetBanking_Flag', 'Avg_days_between_transaction']
['loan_enq', 'account_type', 'gender', 'Income']


In [16]:
df_existing_num=df_existing[numeric]
df_existing_num

,cc_cons,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,...,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active,age,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
0,20014.0,24893.000,378.00,10288.00,29664.00,16291.40,11432.00,2.0,26,10,...,2,65,50836.0,1674.09,35,15.0,1,708.0,0,17.0
1,10173.0,18941.620,966.00,20672.00,287.00,4217.00,885.00,1.0,7,13,...,21,63,23226.0,13043.34,35,15.0,6,123.0,0,14.0
2,16095.0,5678.870,2724.00,1964.50,3933.11,23956.25,5168.00,43.0,32,102,...,7,1,27036.0,25375.27,55,24.5,1,802.0,1,3.0
3,7707.0,30489.500,1236.00,12609.88,9138.14,17521.00,13650.30,53.0,1,50,...,21,83,43037.0,3544.33,29,4.5,6,867.0,1,4.0
4,96408.0,7441.400,6906.04,4364.00,1939.00,2121.00,6829.18,67.0,47,82,...,8,32,32044.0,12780.44,34,12.6,4,802.0,1,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,NaN,4641.500,5979.58,6008.61,4410.00,41743.00,1884.00,26.0,1,4,...,21,50,58037.0,7870.88,35,13.5,1,523.0,0,7.0
19996,NaN,4994.450,2805.37,23214.00,5870.00,1041.00,1008.73,37.0,56,50,...,62,38,24110.0,3668.30,53,9.9,7,723.0,1,2.0
19997,NaN,3262.145,2871.00,7335.25,11774.04,4130.00,1413.00,12.0,77,1,...,2,18,1769.0,5450.34,62,37.8,8,863.0,1,16.0
19998,NaN,12824.010,4866.00,12242.48,284.00,15132.77,37911.00,67.0,1,4,...,117,64,37255.0,59.29,34,12.6,4,882.0,0,13.0


In [17]:
df_existing_cat=df_existing[categorical_col]
df_existing_cat

,loan_enq,account_type,gender,Income
0,Y,current,M,MEDIUM
1,Y,current,M,HIGH
2,Y,current,M,MEDIUM
3,Y,current,F,MEDIUM
4,NaN,current,M,MEDIUM
...,...,...,...,...
19995,Y,current,M,MEDIUM
19996,Y,current,M,MEDIUM
19997,Y,current,M,HIGH
19998,Y,current,M,MEDIUM


In [18]:
def var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  x.std(), x.var(), x.min(), x.dropna().quantile(0.01), x.dropna().quantile(0.05),x.dropna().quantile(0.10),x.dropna().quantile(0.25),x.dropna().quantile(0.50),x.dropna().quantile(0.75), x.dropna().quantile(0.90),x.dropna().quantile(0.95), x.dropna().quantile(0.99),x.max()], 
                  index=['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1' , 'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])

In [19]:
num_summary=df_existing_num.apply(lambda x : var_summary(x)).T
num_summary

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
cc_cons,15000.0,5000.0,8.273659e+08,55157.729867,13364.0000,9.767097e+04,9.539619e+09,0.00,271.9900,1290.85000,2633.600,6768.000,13364.0000,20105.7500,219688.7000,3.098520e+05,3.952905e+05,408382.00
cc_cons_apr,20000.0,0.0,3.571398e+08,17856.990922,10214.6425,2.515888e+04,6.329692e+08,253.00,641.9886,1219.00000,1908.000,4377.000,10214.6425,21189.0000,40640.9000,5.910673e+04,1.252595e+05,441031.33
dc_cons_apr,20000.0,0.0,1.626091e+08,8130.453414,3964.7500,1.457219e+04,2.123487e+08,19.35,153.1286,378.00000,643.850,1624.650,3964.7500,9339.0200,18443.6910,2.783500e+04,6.447075e+04,229255.00
cc_cons_may,19999.0,1.0,3.237511e+08,16188.364717,8635.2300,2.651068e+04,7.028162e+08,157.00,509.4900,1035.77400,1666.328,3635.910,8635.2300,18655.3050,36139.5880,5.396238e+04,1.257354e+05,1117246.31
dc_cons_may,20000.0,0.0,1.540485e+08,7702.427191,3791.7200,1.789273e+04,3.201499e+08,22.00,152.1000,410.97000,731.000,1634.260,3791.7200,8263.5900,16144.7600,2.558400e+04,6.596306e+04,911223.49
cc_cons_jun,20000.0,0.0,2.427161e+08,12135.806021,6656.8000,1.879098e+04,3.531011e+08,65.00,334.0000,825.10100,1367.520,2999.075,6656.8000,14195.7275,26791.1665,3.997968e+04,9.031008e+04,382914.10
dc_cons_jun,19999.0,1.0,1.421871e+08,7109.708341,3326.0000,1.331394e+04,1.772609e+08,62.00,190.9500,390.00000,607.000,1361.100,3326.0000,7458.8200,15689.0000,2.425288e+04,7.256400e+04,233384.00
cc_count_apr,19999.0,1.0,8.321910e+05,41.611631,10.0000,9.783547e+01,9.571780e+03,1.00,1.0000,1.00000,2.000,5.000,10.0000,44.0000,80.0000,1.240000e+02,5.310000e+02,531.00
cc_count_may,20000.0,0.0,6.713820e+05,33.569100,23.0000,5.031692e+01,2.531793e+03,1.00,1.0000,2.00000,3.000,8.000,23.0000,43.0000,83.0000,9.300000e+01,1.280000e+02,601.00
cc_count_jun,20000.0,0.0,8.294040e+05,41.470200,35.0000,4.682566e+01,2.192642e+03,1.00,1.0000,3.00000,4.000,10.000,35.0000,61.0000,84.0000,8.900000e+01,1.350000e+02,653.00


In [20]:
def category(x):
    return pd.Series([x.count(),x.isnull().sum(),x.value_counts(),x.unique()],
                    index=["count","nmiss","columnsnames","unique"])


In [21]:
cat_summary=df_existing.apply(lambda x : category(x)).T

In [22]:
cat_summary

,count,nmiss,columnsnames,unique
cc_cons,15000,5000,cc_cons 6035.0 5 13260.0 5 6682.0 ...,"[20014.0, 10173.0, 16095.0, 7707.0, 96408.0, 1..."
cc_cons_apr,20000,0,cc_cons_apr 4286.860 19 27109.000 18 92...,"[24893.0, 18941.62, 5678.87, 30489.5, 7441.4, ..."
dc_cons_apr,20000,0,dc_cons_apr 8762.00 34 36242.82 33 703....,"[378.0, 966.0, 2724.0, 1236.0, 6906.04, 1597.5..."
cc_cons_may,19999,1,cc_cons_may 3392.26 16 19514.97 15 780....,"[10288.0, 20672.0, 1964.5, 12609.88, 4364.0, 1..."
dc_cons_may,20000,0,dc_cons_may 248.90 37 1005.00 35 1120...,"[29664.0, 287.0, 3933.11, 9138.14, 1939.0, 104..."
cc_cons_jun,20000,0,cc_cons_jun 16527.000 21 16940.535 17 35...,"[16291.4, 4217.0, 23956.25, 17521.0, 2121.0, 1..."
dc_cons_jun,19999,1,dc_cons_jun 21126.00 63 389.00 45 6234...,"[11432.0, 885.0, 5168.0, 13650.3, 6829.18, 628..."
cc_count_apr,19999,1,cc_count_apr 6.0 1746 1.0 1534 2.0 ...,"[2.0, 1.0, 43.0, 53.0, 67.0, 6.0, 10.0, 26.0, ..."
cc_count_may,20000,0,cc_count_may 10 1785 23 1144 3 10...,"[26, 7, 32, 1, 47, 2, 34, 4, 65, 66, 77, 36, 2..."
cc_count_jun,20000,0,cc_count_jun 50 1329 4 1087 5 720 ...,"[10, 13, 102, 50, 82, 59, 5, 84, 87, 67, 9, 19..."


## Filling missing values

In [23]:
df_existing_cat=df_existing_cat.fillna(df_existing_cat.mode().iloc[0])

In [24]:
df_existing_num=df_existing_num.fillna(df_existing_num.mean())

In [25]:
# Find the total number of missing values in the numerical data i.e custdata_df_num
print ("Missing values in numerical data :  ", df_existing_num.isnull().sum().values.sum())
print ("Missing values in categorical data :  ", df_existing_cat.isnull().sum().values.sum())

Missing values in numerical data :   0
Missing values in categorical data :   0


## Labelling categorical column to numerical column

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
for i in df_existing_cat.columns:
    le=LabelEncoder()
    df_existing_cat[i]=le.fit_transform(df_existing_cat[i])

In [28]:
df_existing_cat

,loan_enq,account_type,gender,Income
0,0,0,1,2
1,0,0,1,0
2,0,0,1,2
3,0,0,0,2
4,0,0,1,2
...,...,...,...,...
19995,0,0,1,2
19996,0,0,1,2
19997,0,0,1,0
19998,0,0,1,2


## merge the numerical and categorical dataset

In [29]:
df_existing_final=pd.concat([df_existing_cat,df_existing_num],axis=1)

## Handling outlier treatment

In [30]:
df_existing_final=df_existing_final.clip(lower=df_existing_final.quantile(0.05),upper=df_existing_final.quantile(0.95),axis=1)

In [31]:
df_existing_final.head()

,loan_enq,account_type,gender,Income,cc_cons,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,...,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active,age,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
0,0,0,1,2,20014.0,24893.00,378.00,10288.00,25584.00,16291.40,...,2,65,50836.0,1674.09,35,15.0,1,708.0,0,17.0
1,0,0,1,0,10173.0,18941.62,966.00,20672.00,410.97,4217.00,...,21,63,23226.0,13043.34,35,15.0,6,249.0,0,14.0
2,0,0,1,2,16095.0,5678.87,2724.00,1964.50,3933.11,23956.25,...,7,2,27036.0,25375.27,55,24.5,1,802.0,1,3.0
3,0,0,0,2,7707.0,30489.50,1236.00,12609.88,9138.14,17521.00,...,21,83,43037.0,3544.33,29,4.5,6,867.0,1,4.0
4,0,0,1,2,96408.0,7441.40,6906.04,4364.00,1939.00,2121.00,...,8,32,32044.0,12780.44,34,12.6,4,802.0,1,7.0


## Separating Feature columns

In [32]:
X=df_existing_final.columns.difference(["cc_cons"])
Y=df_existing_final['cc_cons']

## Spliting Train and Test data

In [33]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_existing_final[X],Y,test_size=0.2,random_state=12)

In [34]:
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

16000
4000
16000
4000


## Using DecisionTree for Prediction

In [35]:
from sklearn.tree import DecisionTreeRegressor

In [36]:
model=DecisionTreeRegressor()
model.fit(x_train,y_train)

DecisionTreeRegressor()

In [37]:
pred_y=model.predict(x_test)

In [57]:
from sklearn.metrics import mean_squared_error

In [58]:
from sklearn import metrics

In [64]:
rmspe = np.sqrt(np.mean(np.square(((y_test - pred_y) / y_test)), axis=0))
print(f"Root Mean Square Percentage Error (RMSPE): {round(rmspe,2)}%")

Root Mean Square Percentage Error (RMSPE): 17.83%


In [59]:
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred_y))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred_y))
print('Root Mean Squared Error (RMSE):', metrics.mean_squared_error(y_test, pred_y, squared=False))
print('Mean Absolute Percentage Error (MAPE):', metrics.mean_absolute_percentage_error(y_test, pred_y))
print('Explained Variance Score:', metrics.explained_variance_score(y_test, pred_y))
print('Max Error:', metrics.max_error(y_test, pred_y))
print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, pred_y))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, pred_y))
print('R^2:', metrics.r2_score(y_test, pred_y))
print('Mean Poisson Deviance:', metrics.mean_poisson_deviance(y_test, pred_y))
print('Mean Gamma Deviance:', metrics.mean_gamma_deviance(y_test, pred_y))

Mean Absolute Error (MAE): 71829.90468440001
Mean Squared Error (MSE): 13173279076.35361
Root Mean Squared Error (RMSE): 114774.90612652928
Mean Absolute Percentage Error (MAPE): 5.95675468188753
Explained Variance Score: -1.183545720923183
Max Error: 297209.85
Mean Squared Log Error: 3.921817730370833
Median Absolute Error: 40070.72986666667
R^2: -1.2002291471368967
Mean Poisson Deviance: 174352.4372300877
Mean Gamma Deviance: 9.345346192178024


##  Predicted values for customers where target variable having missing values

In [40]:
missing=df_existing[df_existing['cc_cons'].isna()]

## Cleaning the missing dataset

In [41]:
mis_cat=missing.select_dtypes(include=['object'])
for i in mis_cat:
    le=LabelEncoder()
    missing[i]=le.fit_transform(missing[i])

C:\Users\aazar\AppData\Local\Temp\ipykernel_23796\1804594118.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing[i]=le.fit_transform(missing[i])
C:\Users\aazar\AppData\Local\Temp\ipykernel_23796\1804594118.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing[i]=le.fit_transform(missing[i])
C:\Users\aazar\AppData\Local\Temp\ipykernel_23796\1804594118.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [42]:
missing=missing.fillna(missing.mean())

In [43]:
missing

,cc_cons,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,...,emi_active,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
15000,NaN,2795.450,1290.00,6141.05,676.50,9146.10,15479.00,4.0,10,4,...,1170.49,0,1,37,2,11.9,4,575.0,1,16.000000
15001,NaN,29405.130,1640.00,1670.00,2463.92,10947.50,956.00,3.0,10,4,...,16447.45,0,1,33,2,7.8,6,394.0,0,19.000000
15002,NaN,5985.200,6189.00,2696.09,766.00,716.00,4133.22,7.0,30,5,...,2622.28,0,1,53,1,33.0,10,324.0,1,15.000000
15003,NaN,2105.930,18225.00,34763.56,167.00,4260.27,20185.00,1.0,99,71,...,340.79,0,1,33,2,7.8,5,370.0,1,7.000000
15004,NaN,3269.000,3532.00,3158.40,2699.77,3373.48,5120.00,2.0,10,47,...,2812.20,0,1,62,1,12.6,9,505.0,1,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,NaN,4641.500,5979.58,6008.61,4410.00,41743.00,1884.00,26.0,1,4,...,7870.88,0,1,35,2,13.5,1,523.0,0,7.000000
19996,NaN,4994.450,2805.37,23214.00,5870.00,1041.00,1008.73,37.0,56,50,...,3668.30,0,1,53,2,9.9,7,723.0,1,2.000000
19997,NaN,3262.145,2871.00,7335.25,11774.04,4130.00,1413.00,12.0,77,1,...,5450.34,0,1,62,0,37.8,8,863.0,1,16.000000
19998,NaN,12824.010,4866.00,12242.48,284.00,15132.77,37911.00,67.0,1,4,...,59.29,0,1,34,2,12.6,4,882.0,0,13.000000


In [44]:
#outlier treatment
missing=missing.clip(lower=missing.quantile(0.05),upper=missing.quantile(0.95),axis=1)

In [45]:
missing_nan=missing[missing["cc_cons"].isna()]

In [46]:
#droping cc_cons column which has null values
missing_nan=missing_nan.drop(columns=["cc_cons"],axis=1)

## Training the dataset

In [47]:
missing_nan = missing_nan[x_train.columns]

In [48]:
missing_nan

,Avg_days_between_transaction,Emp_Tenure_Years,Income,NetBanking_Flag,Tenure_with_Bank,account_type,age,card_lim,cc_cons_apr,cc_cons_jun,...,investment_4,loan_enq,max_credit_amount_apr,max_credit_amount_jun,max_credit_amount_may,personal_loan_active,personal_loan_closed,region_code,vehicle_loan_active,vehicle_loan_closed
15000,16.000000,11.9,2,1,4,0,37,125000,2795.450,9146.10000,...,-5715.49023,0,24485.00,25945.92,9750.0,1,1.0,575.0,1,1
15001,19.000000,7.8,2,0,6,0,33,59000,29405.130,10947.50000,...,19453.00000,0,10096.11,12214.00,16967.0,1,1.0,394.0,1,1
15002,15.000000,33.0,1,1,10,0,53,49999,5985.200,812.95000,...,2557.00000,0,14950.00,13215.00,36398.0,1,1.0,324.0,1,1
15003,7.000000,7.8,2,1,5,0,33,520000,2105.930,4260.27000,...,5464.00000,0,142896.00,72317.00,97825.0,1,1.0,370.0,1,1
15004,3.000000,12.6,1,1,9,0,62,152000,3269.000,3373.48000,...,1332.00000,0,14495.00,42344.00,27936.0,1,1.0,505.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,7.000000,13.5,2,0,1,0,35,579000,4641.500,39686.79025,...,553.00000,0,27825.00,58037.00,32944.0,1,1.0,523.0,1,1
19996,2.000000,9.9,2,1,7,0,53,27000,4994.450,1041.00000,...,19453.00000,0,27912.00,24110.00,8850.0,1,1.0,723.0,1,1
19997,16.000000,37.8,0,1,8,0,62,124000,3262.145,4130.00000,...,6885.00000,0,14580.00,1769.00,35819.0,1,1.0,863.0,1,1
19998,13.000000,12.6,2,0,4,0,34,371000,12824.010,15132.77000,...,10455.00000,0,23881.00,37255.00,39083.0,1,1.0,882.0,1,1


## predicting the cc_cons column values

In [49]:
predicted_value=model.predict(missing_nan)

In [50]:
missing.loc[missing["cc_cons"].isna(),"cc_cons"]=predicted_value

In [51]:
missing["cc_cons"]=round(missing['cc_cons'],2)

## final output

In [55]:
missing.reset_index(drop=True)

,cc_cons,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,...,emi_active,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
0,55157.73,2795.450,1290.00,6141.05,676.50,9146.10000,15479.00,4.0,10,4,...,1170.490,0,1,37,2,11.9,4,575.0,1,16.000000
1,55157.73,29405.130,1640.00,1670.00,2463.92,10947.50000,956.00,3.0,10,4,...,16447.450,0,1,33,2,7.8,6,394.0,0,19.000000
2,55157.73,5985.200,6189.00,2696.09,766.00,812.95000,4133.22,7.0,30,5,...,2622.280,0,1,53,1,33.0,10,324.0,1,15.000000
3,55157.73,2105.930,18225.00,34763.56,410.90,4260.27000,20185.00,1.0,93,71,...,340.790,0,1,33,2,7.8,5,370.0,1,7.000000
4,93010.00,3269.000,3532.00,3158.40,2699.77,3373.48000,5120.00,2.0,10,47,...,2812.200,0,1,62,1,12.6,9,505.0,1,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,55157.73,4641.500,5979.58,6008.61,4410.00,39686.79025,1884.00,26.0,2,4,...,7870.880,0,1,35,2,13.5,1,523.0,0,7.000000
4996,36857.00,4994.450,2805.37,23214.00,5870.00,1041.00000,1008.73,37.0,56,50,...,3668.300,0,1,53,2,9.9,7,723.0,1,2.000000
4997,55157.73,3262.145,2871.00,7335.25,11774.04,4130.00000,1413.00,12.0,77,3,...,5450.340,0,1,62,0,37.8,8,863.0,1,16.000000
4998,9604.00,12824.010,4866.00,12242.48,410.90,15132.77000,26187.25,67.0,2,4,...,195.866,0,1,34,2,12.6,4,882.0,0,13.000000
